# AI Career Recommendation System

This notebook contains the complete implementation of the Career Recommendation System built for a college hackathon.

## Sections:
1. Data Loading
2. Data Preprocessing
3. Model Training
4. Evaluation
5. Prediction


In [15]:
"""
Career Recommendation System - ML Web App
==========================================
This script generates synthetic data, trains a RandomForestClassifier,
and creates an interactive Gradio interface for career recommendations.

Author: AI Data Science Assistant
Date: 2025-11-05
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:

# ============================================================================
# SECTION 2: DATA PREPROCESSING & MODEL TRAINING
# ============================================================================

def prepare_and_train_model(df):
    """
    Preprocess data and train RandomForestClassifier.

    Args:
        df (pd.DataFrame): Input dataset

    Returns:
        tuple: (model, encoders, X_test, y_test, feature_names)
    """

    # Separate features and target
    X = df.drop(['Recommended_Course', 'Student_ID'], axis=1)
    y = df['Recommended_Course']

    # Store feature names for later use
    feature_names = X.columns.tolist()

    # Encode categorical features
    encoders = {}
    X_encoded = X.copy()

    for col in ['Interest_Area', 'Preferred_Learning_Type']:
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X[col])
        encoders[col] = le

    # Encode target variable
    target_encoder = LabelEncoder()
    y_encoded = target_encoder.fit_transform(y)
    encoders['target'] = target_encoder

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    # Train Random Forest Classifier
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)


    # Classification report
    report = classification_report(
        y_test, y_pred,
        target_names=target_encoder.classes_,
        zero_division=0
    )

    # Feature importance
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)

    for idx, row in feature_importance.iterrows():

    return model, encoders, X_test, y_test, feature_names

In [ ]:
# ============================================================================
# SECTION 3: GRADIO INTERFACE
# ============================================================================

def create_prediction_function(model, encoders, feature_names):
    """
    Create prediction function for Gradio interface.

    Args:
        model: Trained model
        encoders: Dictionary of label encoders
        feature_names: List of feature names

    Returns:
        function: Prediction function for Gradio
    """
    def predict(interest_area, technical_skill, creativity, analytical_ability,
                learning_type, communication, teamwork):
        """
        Make prediction based on user inputs.

        Returns:
            str: Predicted course with confidence scores
        """
        try:
            # Validate numerical inputs
            scores = [technical_skill, creativity, analytical_ability,
                     communication, teamwork]
            for score in scores:
                if not (4 <= score <= 10):
                    return "❌ Error: All scores must be between 4 and 10"

            # Create input dataframe
            input_data = pd.DataFrame({
                'Interest_Area': [interest_area],
                'Technical_Skill_Score': [technical_skill],
                'Creativity_Score': [creativity],
                'Analytical_Ability': [analytical_ability],
                'Preferred_Learning_Type': [learning_type],
                'Communication_Score': [communication],
                'Teamwork_Score': [teamwork]
            })

            # Encode categorical features
            input_encoded = input_data.copy()
            for col in ['Interest_Area', 'Preferred_Learning_Type']:
                input_encoded[col] = encoders[col].transform(input_data[col])

            # Make prediction
            prediction = model.predict(input_encoded)[0]
            predicted_course = encoders['target'].inverse_transform([prediction])[0]

            # Get prediction probabilities
            proba = model.predict_proba(input_encoded)[0]
            top_3_idx = np.argsort(proba)[-3:][::-1]

            # Format output
            result = f"🎯 **Recommended Course:** {predicted_course}\n\n"
            result += "**Top 3 Predictions:**\n"
            for i, idx in enumerate(top_3_idx, 1):
                course = encoders['target'].inverse_transform([idx])[0]
                confidence = proba[idx] * 100
                result += f"{i}. {course}: {confidence:.2f}%\n"

            return result

        except Exception as e:
            return f"❌ Error: {str(e)}\nPlease check your inputs and try again."

    return predict

In [ ]:
# ============================================================================
# SECTION 3: GRADIO INTERFACE
# ============================================================================

def create_prediction_function(model, encoders, feature_names):
    """
    Create prediction function for Gradio interface.

    Args:
        model: Trained model
        encoders: Dictionary of label encoders
        feature_names: List of feature names

    Returns:
        function: Prediction function for Gradio
    """
    def predict(interest_area, technical_skill, creativity, analytical_ability,
                learning_type, communication, teamwork):
        """
        Make prediction based on user inputs.

        Returns:
            str: Predicted course with confidence scores
        """
        try:
            # Validate numerical inputs
            scores = [technical_skill, creativity, analytical_ability,
                     communication, teamwork]
            for score in scores:
                if not (4 <= score <= 10):
                    return "❌ Error: All scores must be between 4 and 10"

            # Create input dataframe
            input_data = pd.DataFrame({
                'Interest_Area': [interest_area],
                'Technical_Skill_Score': [technical_skill],
                'Creativity_Score': [creativity],
                'Analytical_Ability': [analytical_ability],
                'Preferred_Learning_Type': [learning_type],
                'Communication_Score': [communication],
                'Teamwork_Score': [teamwork]
            })

            # Encode categorical features
            input_encoded = input_data.copy()
            for col in ['Interest_Area', 'Preferred_Learning_Type']:
                input_encoded[col] = encoders[col].transform(input_data[col])

            # Make prediction
            prediction = model.predict(input_encoded)[0]
            predicted_course = encoders['target'].inverse_transform([prediction])[0]

            # Get prediction probabilities
            proba = model.predict_proba(input_encoded)[0]
            top_3_idx = np.argsort(proba)[-3:][::-1]

            # Format output
            result = f"🎯 **Recommended Course:** {predicted_course}\n\n"
            result += "**Top 3 Predictions:**\n"
            for i, idx in enumerate(top_3_idx, 1):
                course = encoders['target'].inverse_transform([idx])[0]
                confidence = proba[idx] * 100
                result += f"{i}. {course}: {confidence:.2f}%\n"

            return result

        except Exception as e:
            return f"❌ Error: {str(e)}\nPlease check your inputs and try again."

    return predict


def build_gradio_interface(model, encoders, feature_names):
    """
    Build and launch Gradio interface.

    Args:
        model: Trained model
        encoders: Dictionary of encoders
        feature_names: List of feature names
    """

    # Get options for categorical inputs
    interest_areas = encoders['Interest_Area'].classes_.tolist()
    learning_types = encoders['Preferred_Learning_Type'].classes_.tolist()

    # Create prediction function
    predict_fn = create_prediction_function(model, encoders, feature_names)

    # Build interface
    interface = gr.Interface(
        fn=predict_fn,
        inputs=[
            gr.Dropdown(
                choices=interest_areas,
                label="🎓 Interest Area",
                value=interest_areas[0]
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="💻 Technical Skill Score (4-10)",
                value=7
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="🎨 Creativity Score (4-10)",
                value=7
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="🧠 Analytical Ability (4-10)",
                value=7
            ),
            gr.Radio(
                choices=learning_types,
                label="📚 Preferred Learning Type",
                value=learning_types[0]
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="💬 Communication Score (4-10)",
                value=7
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="🤝 Teamwork Score (4-10)",
                value=7
            )
        ],
        outputs=gr.Textbox(label="📋 Prediction Result", lines=8),
        title="🎓 Career Recommendation System",
        description="""
        **Welcome to the AI-Powered Career Recommendation System!**

        Enter your skills and preferences to get personalized course recommendations.
        The model uses Random Forest algorithm trained on 1000+ student profiles.
        """,
        theme=gr.themes.Soft(),
        examples=[
            ["Web Development", 9, 7, 8, "Practical", 6, 7],
            ["Artificial Intelligence", 8, 6, 9, "Theory", 7, 6],
            ["UI/UX Design", 6, 10, 7, "Practical", 8, 8],
            ["Data Science", 9, 5, 10, "Theory", 7, 6]
        ]
    )


    return interface

In [ ]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution function."""

    # Step 1: Generate synthetic data
    df = generate_synthetic_data(n_rows=1000)

    # Step 2: Train model
    model, encoders, X_test, y_test, feature_names = prepare_and_train_model(df)

    # Step 3: Build and launch Gradio interface
    interface = build_gradio_interface(model, encoders, feature_names)

    # Launch the interface

    interface.launch(
        share=False,  # Set to True to create public link
        debug=False,
        show_error=True
    )


if __name__ == "__main__":
    main()



               CAREER RECOMMENDATION ML SYSTEM

GENERATING SYNTHETIC DATA
✓ Generated 1000 rows with 8 feature columns + 1 target column
✓ Columns: Student_ID, Interest_Area, Technical_Skill_Score, Creativity_Score, Analytical_Ability, Preferred_Learning_Type, Communication_Score, Teamwork_Score, Recommended_Course

  Student_ID               Interest_Area  Technical_Skill_Score  \
0      S0001                Architecture                      4   
1      S0002           Digital Marketing                      6   
2      S0003             Cloud Computing                      8   
3      S0004          Filmmaking & Media                      8   
4      S0005       Business & Management                      7   
5      S0006  Influencing / Social Media                      9   
6      S0007             Cloud Computing                      4   
7      S0008     Artificial Intelligence                      4   
8      S0009                Architecture                      5   
9      S001

<IPython.core.display.Javascript object>

In [ ]:
"""
Career Recommendation System - ML Web App
==========================================
This script generates synthetic data, trains a RandomForestClassifier,
saves the model, and creates an interactive Gradio interface.

Author: AI Data Science Assistant
Date: 2025-11-05
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import gradio as gr
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)


# ============================================================================
# SECTION 1: SYNTHETIC DATA GENERATION
# ============================================================================

def generate_synthetic_data(n_rows=1000):
    """
    Generate synthetic career recommendation dataset.

    Args:
        n_rows (int): Number of rows to generate

    Returns:
        pd.DataFrame: Synthetic dataset with 8 columns
    """

    # Define categorical options based on original dataset
    interest_areas = [
        "Web Development", "UI/UX Design", "Journalism", "Digital Marketing",
        "Artificial Intelligence", "Fashion & Design", "Architecture",
        "Influencing / Social Media", "Data Science", "Cybersecurity",
        "Business & Management", "Finance & Accounting", "Cloud Computing",
        "Psychology", "Filmmaking & Media"
    ]

    learning_types = ["Practical", "Theory"]

    # Course recommendations mapped to interest areas
    course_mapping = {
        "Web Development": ["Web Design", "Front-End Development", "Full-Stack Development"],
        "UI/UX Design": ["Interaction Design", "User Experience", "Graphic Design"],
        "Journalism": ["Media Ethics", "Investigative Reporting", "Content Writing"],
        "Digital Marketing": ["SEO Management", "Marketing", "Social Media Marketing"],
        "Artificial Intelligence": ["Machine Learning", "AI Fundamentals", "Deep Learning"],
        "Fashion & Design": ["Fashion Styling", "Textile Design", "Fashion Marketing"],
        "Architecture": ["Sustainable Architecture", "Interior Architecture", "Urban Design"],
        "Influencing / Social Media": ["Personal Branding", "Influencer Strategy", "Content Creation"],
        "Data Science": ["Data Analytics", "Business Analytics", "Statistical Analysis"],
        "Cybersecurity": ["Information Security", "Network Security", "Ethical Hacking"],
        "Business & Management": ["Operations Management", "Entrepreneurship", "Project Management"],
        "Finance & Accounting": ["Investment Banking", "Financial Analysis", "Corporate Finance"],
        "Cloud Computing": ["Cloud Architecture", "Cloud Infrastructure", "DevOps"],
        "Psychology": ["Counselling", "Behavioral Studies", "Clinical Psychology"],
        "Filmmaking & Media": ["Video Content Creation", "Script Writing", "Cinematography", "Video Editing"]
    }

    # Generate data
    data = {
        'Student_ID': [f'S{str(i+1).zfill(4)}' for i in range(n_rows)],
        'Interest_Area': np.random.choice(interest_areas, n_rows),
        'Technical_Skill_Score': np.random.randint(4, 11, n_rows),
        'Creativity_Score': np.random.randint(4, 11, n_rows),
        'Analytical_Ability': np.random.randint(4, 11, n_rows),
        'Preferred_Learning_Type': np.random.choice(learning_types, n_rows),
        'Communication_Score': np.random.randint(4, 11, n_rows),
        'Teamwork_Score': np.random.randint(4, 11, n_rows)
    }

    # Generate recommended courses based on interest area with some logic
    recommended_courses = []
    for i in range(n_rows):
        interest = data['Interest_Area'][i]
        tech_score = data['Technical_Skill_Score'][i]
        creativity = data['Creativity_Score'][i]

        # Select appropriate course based on interest area and skills
        possible_courses = course_mapping[interest]

        # Add some logic to course selection
        if len(possible_courses) > 1:
            if tech_score >= 8:
                course = possible_courses[min(0, len(possible_courses)-1)]
            elif creativity >= 8:
                course = possible_courses[min(1, len(possible_courses)-1)]
            else:
                course = np.random.choice(possible_courses)
        else:
            course = possible_courses[0]

        recommended_courses.append(course)

    data['Recommended_Course'] = recommended_courses

    df = pd.DataFrame(data)


    # Display statistics
              'Communication_Score', 'Teamwork_Score']].describe())

    return df


# ============================================================================
# SECTION 2: DATA PREPROCESSING & MODEL TRAINING
# ============================================================================

def prepare_and_train_model(df):
    """
    Preprocess data and train RandomForestClassifier.

    Args:
        df (pd.DataFrame): Input dataset

    Returns:
        tuple: (model, encoders, X_test, y_test, feature_names)
    """

    # Separate features and target
    X = df.drop(['Recommended_Course', 'Student_ID'], axis=1)
    y = df['Recommended_Course']

    # Store feature names for later use
    feature_names = X.columns.tolist()

    # Encode categorical features
    encoders = {}
    X_encoded = X.copy()

    for col in ['Interest_Area', 'Preferred_Learning_Type']:
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X[col])
        encoders[col] = le

    # Encode target variable
    target_encoder = LabelEncoder()
    y_encoded = target_encoder.fit_transform(y)
    encoders['target'] = target_encoder

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    # Train Random Forest Classifier
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)


    # Classification report
    report = classification_report(
        y_test, y_pred,
        target_names=target_encoder.classes_,
        zero_division=0
    )

    # Feature importance
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)

    for idx, row in feature_importance.iterrows():

    return model, encoders, X_test, y_test, feature_names


# ============================================================================
# SECTION 3: SAVE MODEL AND CREATE DEPLOYMENT FILES
# ============================================================================

def save_model_and_files(model, encoders, feature_names):
    """
    Save trained model, encoders, and create deployment files.

    Args:
        model: Trained RandomForestClassifier
        encoders: Dictionary of label encoders
        feature_names: List of feature names
    """

    # Save model and encoders as pickle file
    model_data = {
        'model': model,
        'encoders': encoders,
        'feature_names': feature_names
    }

    with open('model.pkl', 'wb') as f:
        pickle.dump(model_data, f)



In [16]:
def create_prediction_function(model, encoders, feature_names):
    """
    Create prediction function for Gradio interface.

    Args:
        model: Trained model
        encoders: Dictionary of label encoders
        feature_names: List of feature names

    Returns:
        function: Prediction function for Gradio
    """
    def predict(interest_area, technical_skill, creativity, analytical_ability,
                learning_type, communication, teamwork):
        """
        Make prediction based on user inputs.

        Returns:
            str: Predicted course with confidence scores
        """
        try:
            # Validate numerical inputs
            scores = [technical_skill, creativity, analytical_ability,
                     communication, teamwork]
            for score in scores:
                if not (4 <= score <= 10):
                    return "❌ Error: All scores must be between 4 and 10"

            # Create input dataframe
            input_data = pd.DataFrame({
                'Interest_Area': [interest_area],
                'Technical_Skill_Score': [technical_skill],
                'Creativity_Score': [creativity],
                'Analytical_Ability': [analytical_ability],
                'Preferred_Learning_Type': [learning_type],
                'Communication_Score': [communication],
                'Teamwork_Score': [teamwork]
            })

            # Encode categorical features
            input_encoded = input_data.copy()
            for col in ['Interest_Area', 'Preferred_Learning_Type']:
                input_encoded[col] = encoders[col].transform(input_data[col])

            # Make prediction
            prediction = model.predict(input_encoded)[0]
            predicted_course = encoders['target'].inverse_transform([prediction])[0]

            # Get prediction probabilities
            proba = model.predict_proba(input_encoded)[0]
            top_3_idx = np.argsort(proba)[-3:][::-1]

            # Format output
            result = f"🎯 **Recommended Course:** {predicted_course}\n\n"
            result += "**Top 3 Predictions:**\n"
            for i, idx in enumerate(top_3_idx, 1):
                course = encoders['target'].inverse_transform([idx])[0]
                confidence = proba[idx] * 100
                result += f"{i}. {course}: {confidence:.2f}%\n"

            return result

        except Exception as e:
            return f"❌ Error: {str(e)}\nPlease check your inputs and try again."

    return predict


def build_gradio_interface(model, encoders, feature_names):
    """
    Build and launch Gradio interface.

    Args:
        model: Trained model
        encoders: Dictionary of encoders
        feature_names: List of feature names
    """

    # Get options for categorical inputs
    interest_areas = encoders['Interest_Area'].classes_.tolist()
    learning_types = encoders['Preferred_Learning_Type'].classes_.tolist()

    # Create prediction function
    predict_fn = create_prediction_function(model, encoders, feature_names)

    # Build interface
    interface = gr.Interface(
        fn=predict_fn,
        inputs=[
            gr.Dropdown(
                choices=interest_areas,
                label="🎓 Interest Area",
                value=interest_areas[0]
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="💻 Technical Skill Score (4-10)",
                value=7
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="🎨 Creativity Score (4-10)",
                value=7
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="🧠 Analytical Ability (4-10)",
                value=7
            ),
            gr.Radio(
                choices=learning_types,
                label="📚 Preferred Learning Type",
                value=learning_types[0]
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="💬 Communication Score (4-10)",
                value=7
            ),
            gr.Slider(
                minimum=4, maximum=10, step=1,
                label="🤝 Teamwork Score (4-10)",
                value=7
            )
        ],
        outputs=gr.Textbox(label="📋 Prediction Result", lines=8),
        title="🎓 Career Recommendation System",
        description="""
        **Welcome to the AI-Powered Career Recommendation System!**

        Enter your skills and preferences to get personalized course recommendations.
        The model uses Random Forest algorithm trained on 1000+ student profiles.
        """,
        theme=gr.themes.Soft(),
        examples=[
            ["Web Development", 9, 7, 8, "Practical", 6, 7],
            ["Artificial Intelligence", 8, 6, 9, "Theory", 7, 6],
            ["UI/UX Design", 6, 10, 7, "Practical", 8, 8],
            ["Data Science", 9, 5, 10, "Theory", 7, 6]
        ]
    )


    return interface


# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution function."""

    # Step 1: Generate synthetic data
    df = generate_synthetic_data(n_rows=1000)

    # Step 2: Train model
    model, encoders, X_test, y_test, feature_names = prepare_and_train_model(df)

    # Step 3: Save model and create deployment files
    save_model_and_files(model, encoders, feature_names)

    # Step 4: Build and launch Gradio interface
    interface = build_gradio_interface(model, encoders, feature_names)

    # Launch the interface

    interface.launch(
        share=False,  # Set to True to create public link
        debug=False,
        show_error=True
    )


if __name__ == "__main__":
    main()



               CAREER RECOMMENDATION ML SYSTEM

GENERATING SYNTHETIC DATA
✓ Generated 1000 rows with 8 feature columns + 1 target column
✓ Columns: Student_ID, Interest_Area, Technical_Skill_Score, Creativity_Score, Analytical_Ability, Preferred_Learning_Type, Communication_Score, Teamwork_Score, Recommended_Course

  Student_ID               Interest_Area  Technical_Skill_Score  \
0      S0001                Architecture                      4   
1      S0002           Digital Marketing                      6   
2      S0003             Cloud Computing                      8   
3      S0004          Filmmaking & Media                      8   
4      S0005       Business & Management                      7   
5      S0006  Influencing / Social Media                      9   
6      S0007             Cloud Computing                      4   
7      S0008     Artificial Intelligence                      4   
8      S0009                Architecture                      5   
9      S001

<IPython.core.display.Javascript object>